In [ ]:
import os
from pathlib import Path
import torch
from tqdm.auto import tqdm
from torchvision.transforms import functional as TF
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

from data import PairedSRDataset, build_loader
from original_model import SRResNet
from metrics import psnr, ssim

# Config
cfg = {
    'scale': 4,
    'hr_crop': 128,
    'paths': {
        'train_lr': 'train/train_lr',
        'train_hr': 'train/train_hr',
        'val_lr': 'val/val_lr',
        'val_hr': 'val/val_hr',
        'test_lr': 'test/test_lr',
        'test_hr': 'test/test_hr',
    },
}

# Resolve base dir so notebook can run from any cwd
base_dir = None
cwd = Path.cwd().resolve()
for parent in [cwd] + list(cwd.parents):
    if (parent / 'train' / 'train_lr').is_dir():
        base_dir = parent
        break
if base_dir is None:
    raise FileNotFoundError(f"Cannot find 'train/train_lr' from cwd: {cwd}")

cfg['paths']['train_lr'] = str(base_dir / 'train' / 'train_lr')
cfg['paths']['train_hr'] = str(base_dir / 'train' / 'train_hr')
cfg['paths']['val_lr'] = str(base_dir / 'val' / 'val_lr')
cfg['paths']['val_hr'] = str(base_dir / 'val' / 'val_hr')
cfg['paths']['test_lr'] = str(base_dir / 'test' / 'test_lr')
cfg['paths']['test_hr'] = str(base_dir / 'test' / 'test_hr')

# Weight path: prefer repo-local weights; also support Kaggle /kaggle/working layout
candidates = [
    base_dir / 'weights' / 'best_srresnet.pth',
    Path('/kaggle/working') / 'Cuoi_Ki_DL' / 'weights' / 'best_srresnet.pth',
]
weight_path = None
for p in candidates:
    if p.is_file():
        weight_path = str(p)
        break
if weight_path is None:
    raise FileNotFoundError(f"Weight not found. Checked: {[str(p) for p in candidates]}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_srresnet(weight_path, scale, device):
    model = SRResNet(upscale=scale).to(device)
    ckpt = torch.load(weight_path, map_location=device)
    if isinstance(ckpt, dict) and any(k in ckpt for k in ['state_dict', 'model', 'generator']):
        if 'state_dict' in ckpt:
            state = ckpt['state_dict']
        elif 'model' in ckpt:
            state = ckpt['model']
        else:
            state = ckpt['generator']
    else:
        state = ckpt

    # Strip common prefixes if present
    if isinstance(state, dict):
        if any(k.startswith('module.') for k in state):
            state = {k.replace('module.', '', 1): v for k, v in state.items()}
        if any(k.startswith('_orig_mod.') for k in state):
            state = {k.replace('_orig_mod.', '', 1): v for k, v in state.items()}

    missing, unexpected = model.load_state_dict(state, strict=False)
    if missing or unexpected:
        print('Missing keys:', missing)
        print('Unexpected keys:', unexpected)
    model.eval()
    return model

model = load_srresnet(weight_path, cfg['scale'], device)

# Block 1: evaluate metrics on test set
_, test_loader = build_loader(
    cfg['paths']['test_lr'], cfg['paths']['test_hr'],
    scale=cfg['scale'], hr_crop=cfg['hr_crop'],
    batch_size=1, num_workers=4, train=False
)

psnr_vals = []
ssim_vals = []
with torch.no_grad():
    for lr, hr in tqdm(test_loader, desc='Test'):
        lr = lr.to(device)
        hr = hr.to(device)
        sr = model(lr)
        psnr_vals.append(psnr(sr, hr))
        ssim_vals.append(ssim(sr, hr))

avg_psnr = sum(psnr_vals) / max(len(psnr_vals), 1)
avg_ssim = sum(ssim_vals) / max(len(ssim_vals), 1)
print(f'Test PSNR: {avg_psnr:.4f} dB')
print(f'Test SSIM: {avg_ssim:.4f}')


In [ ]:
# Block 2: save SR images for test/train/val (batched + JPG)
from torch.utils.data import DataLoader

class PairedSRDatasetWithPaths(PairedSRDataset):
    def __getitem__(self, idx):
        lr_tensor, hr_tensor = super().__getitem__(idx)
        lr_path, hr_path = self.pairs[idx]
        return lr_tensor, hr_tensor, lr_path, hr_path

def save_sr_split_batched(split_name, lr_dir, hr_dir, out_dir, batch_size=32, num_workers=4, jpg_quality=95):
    os.makedirs(out_dir, exist_ok=True)
    dataset = PairedSRDatasetWithPaths(lr_dir, hr_dir, scale=cfg['scale'], hr_crop=cfg['hr_crop'], train=False)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=num_workers > 0,
    )

    with torch.no_grad():
        for lr_batch, _, lr_paths, hr_paths in tqdm(loader, desc=f'Saving {split_name}'):
            lr_batch = lr_batch.to(device)
            sr_batch = model(lr_batch).cpu()
            for i in range(sr_batch.size(0)):
                base = os.path.splitext(os.path.basename(hr_paths[i]))[0]
                if not base:
                    base = os.path.splitext(os.path.basename(lr_paths[i]))[0]
                out_path = os.path.join(out_dir, f"{base}_sr.jpg")
                img = TF.to_pil_image(sr_batch[i])
                img.save(out_path, format='JPEG', quality=jpg_quality, optimize=True)

sr_root = base_dir / 'sr'
save_sr_split_batched('test', cfg['paths']['test_lr'], cfg['paths']['test_hr'], str(sr_root / 'test'))
save_sr_split_batched('train', cfg['paths']['train_lr'], cfg['paths']['train_hr'], str(sr_root / 'train'))
save_sr_split_batched('val', cfg['paths']['val_lr'], cfg['paths']['val_hr'], str(sr_root / 'val'))

print('Done saving SR images.')
